In [38]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 83.5 MB/s eta 0:00:00


# Importing Libraries

In [46]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import ast
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import numpy as np
import faiss
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Loading the data

In [2]:
df = pd.read_csv("/content/movies_metadata.csv")
print(df.shape)
df.head()

(14171, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14171 entries, 0 to 14170
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  14171 non-null  bool   
 1   belongs_to_collection  1898 non-null   object 
 2   budget                 14171 non-null  int64  
 3   genres                 14171 non-null  object 
 4   homepage               1580 non-null   object 
 5   id                     14171 non-null  int64  
 6   imdb_id                14168 non-null  object 
 7   original_language      14171 non-null  object 
 8   original_title         14171 non-null  object 
 9   overview               14119 non-null  object 
 10  popularity             14171 non-null  float64
 11  poster_path            14130 non-null  object 
 12  production_companies   14171 non-null  object 
 13  production_countries   14171 non-null  object 
 14  release_date           14165 non-null  object 
 15  re

In [5]:
df.isnull().sum()

,0
adult,0
belongs_to_collection,12273
budget,0
genres,0
homepage,12591
id,0
imdb_id,3
original_language,0
original_title,0
overview,52


# Preprocessing and Cleaning data

In [6]:
df.duplicated().sum()

np.int64(4)

In [7]:
df = df.drop_duplicates().reset_index(drop=True)

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [10]:
df = df[['title', 'overview', 'genres', 'tagline', 'vote_average', 'popularity']]

In [11]:
df.head()

,title,overview,genres,tagline,vote_average,popularity
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,7.7,21.946943
1,Jumanji,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Roll the dice and unleash the excitement!,6.9,17.015539
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Still Yelling. Still Fighting. Still Ready for...,6.5,11.712900
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Friends are the people who let you be yourself...,6.1,3.859495
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]",Just When His World Is Back To Normal... He's ...,5.7,8.387519


In [12]:
df = df.dropna(subset=['title'])

In [13]:
df.isnull().sum()

,0
title,0
overview,52
genres,0
tagline,5008
vote_average,0
popularity,0


In [14]:
df['overview'] = df['overview'].fillna('Not available')

In [15]:
df['overview'].sample(10)

,overview
8733,"Bowie, a youthful convicted murderer, and bank..."
13546,"In the summer of 1987, a college graduate take..."
2059,Josh Miller (Tom Everett Scott) is a studious ...
12483,"Phar Lap, the big bold chestnut reigned as the..."
3971,"An all-Irish cast (including Donal McCann, Rac..."
7768,"In a story told in narrative flashbacks, a you..."
11673,A biographical portrait of a pre-fame Jane Aus...
854,A shipwrecked sailor stumbles upon a mysteriou...
9924,"A young and idealistic woman, who has adopted ..."
8913,"Dominick and Eugene are twins, but Dominick is..."


In [16]:
df['genres'] = df['genres'].apply(lambda x: ' '.join([genre['name'] for genre in ast.literal_eval(x)]))

In [17]:
df['genres']

,genres
0,Animation Comedy Family
1,Adventure Fantasy Family
2,Romance Comedy
3,Comedy Drama Romance
4,Comedy
...,...
14161,Horror Mystery
14162,Action Thriller Science Fiction
14163,Animation Comedy Family
14164,Romance Drama


In [18]:
df['tagline'] = df['tagline'].fillna('Not available')

In [19]:
df['tagline'].sample(10)

,tagline
9309,Not available
12968,Not available
3873,The Mountain Will Decide.
5664,Attitude meets espionage.
5793,Not available
300,Not available
7274,Everyone has a past. Everyone has a secret.
3593,An earthquake is about to hit L.A. It's called...
2686,A business on the line. A friendship on the ed...
13325,Not available


In [20]:
df.isnull().sum()

,0
title,0
overview,0
genres,0
tagline,0
vote_average,0
popularity,0


In [21]:
df['tags'] = df['overview'] + " "+ df['genres'] + " " + df['tagline']

In [22]:
df.head()

,title,overview,genres,tagline,vote_average,popularity,tags
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Animation Comedy Family,Not available,7.7,21.946943,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...,Adventure Fantasy Family,Roll the dice and unleash the excitement!,6.9,17.015539,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Romance Comedy,Still Yelling. Still Fighting. Still Ready for...,6.5,11.712900,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Comedy Drama Romance,Friends are the people who let you be yourself...,6.1,3.859495,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,Just When His World Is Back To Normal... He's ...,5.7,8.387519,Just when George Banks has recovered from his ...


In [26]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [27]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [28]:
def preprocess_text(text):
  #lowercase
  text = str(text).lower()
  #removed the punctuations
  text = re.sub(r'[^a-zA-Z\s]', '', text)

  words = text.split()
  #remove stopwords
  words = [word for word in words if word not in stop_words]

  #lemmatize
  words = [lemmatizer.lemmatize(word) for word in words]

  return " ".join(words)


In [29]:
df['tags'] = df['tags'].apply(preprocess_text)

In [30]:
df = df.reset_index(drop = True)

In [31]:
indices = pd.Series(df.index,index = df['title']).drop_duplicates()
indices

,0
title,
Toy Story,0
Jumanji,1
Grumpier Old Men,2
Waiting to Exhale,3
Father of the Bride Part II,4
...,...
The Final Destination,14161
Gamer,14162
Cloudy with a Chance of Meatballs,14163


# Engine 1

In [33]:
model = SentenceTransformer("all-MiniLM-L6-v2")

texts = df['tags'].tolist()
embeddings = model.encode(texts, show_progress_bar=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/443 [00:00<?, ?it/s]

In [35]:
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)


In [40]:
dimension = embeddings.shape[1]

index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

print("Total vectors in index:", index.ntotal)

Total vectors in index: 14166


In [44]:
def recommend_movies(movie_title, df, model, index, top_k=10):
    if movie_title not in df['title'].values:
        return f"Movie '{movie_title}' not found in dataset."

    idx = df[df['title'] == movie_title].index[0]
    query_text = df.loc[idx, 'tags']

    query_embedding = model.encode([query_text])
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)
    distances, indices = index.search(query_embedding, top_k + 1)
    recommended_indices = indices[0][1:]  # skip itself
    recommendations = df.iloc[recommended_indices][['title']]

    return recommendations


In [45]:
print(recommend_movies("Toy Story", df, model, index, top_k=5))

                            title
2996                  Toy Story 2
10975  Keeping Up with the Steins
3142                      Singles
4562      The Hotel New Hampshire
999                  Mary Poppins


# Engine 2

In [47]:
tfidf = TfidfVectorizer(
    max_df=0.8,
    min_df=5,
    stop_words='english',
    ngram_range=(1,2)
)

tfidf_matrix = tfidf.fit_transform(df['tags'])

In [48]:
def recommend_movies_tfidf(movie_title, df, tfidf_matrix, tfidf, top_k=10):
    if movie_title not in df['title'].values:
        return f"Movie '{movie_title}' not found in dataset."

    idx = df[df['title'] == movie_title].index[0]

    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_indices = [i[0] for i in sim_scores[1:top_k+1]]

    return df.iloc[sim_indices][['title']]


In [49]:
print(recommend_movies_tfidf("Toy Story", df, tfidf_matrix, tfidf, top_k=5))

                        title
2996              Toy Story 2
11396  For Your Consideration
6434   What's Up, Tiger Lily?
1071    Rebel Without a Cause
485                    Malice


# Both engines

In [50]:
def recommend(movie_title, engine="embedding", top_k=10):
    if engine == "embedding":
        return recommend_movies(movie_title, df, model, index, top_k)
    elif engine == "tfidf":
        return recommend_movies_tfidf(movie_title, df, tfidf_matrix, tfidf, top_k)
    else:
        return "Invalid engine selected"


In [51]:
print(recommend("Toy Story", engine="embedding", top_k=5))
print(recommend("Toy Story", engine="tfidf", top_k=5))

                            title
2996                  Toy Story 2
10975  Keeping Up with the Steins
3142                      Singles
4562      The Hotel New Hampshire
999                  Mary Poppins
                        title
2996              Toy Story 2
11396  For Your Consideration
6434   What's Up, Tiger Lily?
1071    Rebel Without a Cause
485                    Malice


# Saving files

In [52]:
pickle.dump(tfidf, open("tfidf_vectorizer.pkl", "wb"))
pickle.dump(tfidf_matrix, open("tfidf_matrix.pkl", "wb"))
print("tfidf files saved succesfully.")

tfidf files saved succesfully.


In [53]:
faiss.write_index(index, "movie_faiss.index")
np.save("movie_embeddings.npy", embeddings)
print("faiss & embeddings files saved succesfully.")

faiss & embeddings files saved succesfully.
